### **Players implementation**



To implement the players, I use a genotype containing 8 genes ranging from 0 to 7.


*   genotype[0]: Tendency to flee from an infected area (0 = low to 1 = high)
*   genotype[1]: Tendency to help other players (0 = low to 1 = high)
*   genotype[2]: Tendency to spread the virus (0 = high to 1 = low)

*   genotype[3]: Tendency to explore infected areas (0 = low, 1 = high)

*   genotype[4]: Preference for self-treatment (0 = low, 1 = high)
*   genotype[5]: Virus resistance/level (0 = low, 1 = high)


*   genotype[6]: Movement speed (0 = slow, 1 = fast)


*   genotype[7]: Location (0 = move towards populated areas to 1 = move towards deserted areas)











There are 7 classes, each with different levels of health and a position in a map.



In [1]:
import random
import numpy as np
classPlayer=['warrior','paladin','hunter','rogue','priest','warlock','mage','shaman','druid']



class Player:
    def __init__(self, genotype_length):
        self.genotype = [round(random.random(), 2) for _ in range(genotype_length)]
        #genotype[5]= 0 à 0,6
        self.genotype[5] = round(random.random() * 0.6, 2)
        self.location = None
        self.infected = False

        self.classplayer=classPlayer[random.randint(0,8)]
        self.health = self.calculate_health()
    def calculate_health(self):
        if self.classplayer=='warrior':
            return 150.51*self.genotype[5]*100+120
        if self.classplayer=='paladin':
            return 142.03*self.genotype[5]*100+120
        if self.classplayer=='hunter':
            return 100*self.genotype[5]*100+100
        if self.classplayer=='rogue':
            return 83.05*self.genotype[5]*100+100
        if self.classplayer=='priest':
            return 74.92*self.genotype[5]*100+80
        if self.classplayer=='warlock':
            return 71.53*self.genotype[5]*100+80
        if self.classplayer=='mage':
            return 71.53*self.genotype[5]*100+80
        if self.classplayer=='shaman':
            return 133.56*self.genotype[5]*100+120
        if self.classplayer=='druid':
            return 125.08*self.genotype[5]*100+120


    def move(self, new_location):
        self.location = new_location

    def spread_virus(self):
        pass

    def heal(self):
        pass

def resistanceVirusInfecte():
    res = round(random.random(), 2)

    while res < 0.3:
        res = round(random.random(), 2)
    return res

### **Map implementation**

We have decided to use a matrix to model the World of Warcraft map. At each step, we will see the players move around it.

In [3]:
import numpy as np

class Zone:
    def __init__(self, center, size):
        self.center = center
        self.size = size

class WoWMap:
    def __init__(self, num_zones, zone_size):
        self.num_zones = num_zones
        self.zone_size = zone_size
        self.zones = []
        self.grid = [[0 for _ in range(num_zones * zone_size)] for _ in range(num_zones * zone_size)]

        for i in range(num_zones):
            for j in range(num_zones):
                zone_center = (i * zone_size + zone_size // 2, j * zone_size + zone_size // 2)
                self.zones.append(Zone(zone_center, zone_size))

    def generate_map(self):
        for i in range(len(self.grid)):
            for j in range(len(self.grid[0])):
                if self.is_in_zone(i, j):
                    self.grid[i][j] = 1
                else:
                    self.grid[i][j] = 0

    def print_map(self):
        for row in self.grid:
            for cell in row:
                if cell == 1:
                    print('O', end=' ')
                elif cell == 2:
                    #infected player
                    print('\033[91m' + 'P' + '\033[0m', end=' ')
                elif cell == 3:
                    #non-infected player
                    print('\033[94m' + 'P' + '\033[0m', end=' ')
                elif cell == 4:
                    print('\033[93m' + 'N' + '\033[0m', end=' ')
                elif cell == 5:
                    print('\033[92m' + 'N' + '\033[0m', end=' ')
                else:
                    print('.', end=' ')
            print()

    def is_in_zone(self, x, y):
        for zone in self.zones:
            dx = x - zone.center[0]
            dy = y - zone.center[1]
            if dx ** 2 + dy ** 2 <= (zone.size // 2) ** 2:
                return True
        return False

    def place_player(self, x, y, player_type):
        if self.grid[x][y] == 1:
            self.grid[x][y] = player_type
            return True
        return False


### **Implementation of the genetic algorithm**

**The first step**

As a model, we will work with 100 randomly generated players, 4 of whom are infected, they are the "patient zero."

There will be 4 NPCs who can have the disease and therefore can be asymptomatic carriers.

We will work on stages. At each stage, an infected person can:



*   Either lose HP and die if their health is below 0.
*   Or die because they have not reached level 30.

The Paladin, Druid, Shaman, and Priest classes can heal themselves and others.

And people can move around the map.


In [11]:
import random
import numpy as np
import matplotlib.pyplot as plt



# Map create
wow_map = WoWMap(3, 10)
wow_map.generate_map()
# Find valid positions with 1s on the map
valid_positions = []
for i in range(wow_map.num_zones * wow_map.zone_size):
    for j in range(wow_map.num_zones * wow_map.zone_size):
        if wow_map.grid[i][j] == 1:
            valid_positions.append((i, j))
#gnotype size
genotype_length = 8

length_population = 100
population = [Player(genotype_length) for _ in range(length_population)]
number_infected = 4
for i in range(number_infected):
    population[i].infected = True
    population[i].genotype[5] = 0.6
    population[i].health = Player.calculate_health(population[i])

# creation of the players
length_npc_population = 8
npc_population = [Player(genotype_length) for _ in range(length_npc_population)]
for npc in npc_population:
    npc.infected = False
    npc.genotype = [0] * genotype_length
    npc.health = 100
# Place players randomly on the map
def get_valid_position():
    valid_position = random.choice(valid_positions)
    valid_positions.remove(valid_position)
    return valid_position

# Place players on the map
def player_position():
    for player in population:
        player_position = get_valid_position()
        player.location = player_position
        wow_map.grid[player_position[0]][player_position[1]] = 2 if player.infected else 3
# Place non-players on the map
def npc_position():
    for npc in npc_population:
        npc_position = get_valid_position()
        npc.location = npc_position
        wow_map.grid[npc_position[0]][npc_position[1]] = 4 if npc.infected else 5

player_position()
npc_position()

def afficher_player():
    for player in population:
        if wow_map.grid[player.location[0]][player.location[1]] == 2:
            print('\033[91m', end='')  #Displaying infected players in red
        else:
            print('\033[0m', end='')  # Resetting the text color for non-infected players

        print(player.genotype)

        print('\033[0m', end='')
        print("Santé: ", player.health)
        print("role: ", player.classplayer)
        print('-------------------')


def infect_nearby_players_npc(infect_player):
    for i in range(-2, 3):
        for j in range(-2, 3):
            # check valid position
            if 0 <= infect_player.location[0] + i < wow_map.num_zones * wow_map.zone_size and \
                    0 <= infect_player.location[1] + j < wow_map.num_zones * wow_map.zone_size:

                # Check if a player is present in the cell.
                for player in population:
                    if player.location[0] == infect_player.location[0] + i and \
                            player.location[1] == infect_player.location[1] + j:
                        # Infecter le joueur si la case est occupée par un joueur sain
                        if not player.infected:
                            player.infected = True
                            player.genotype[5] = resistanceVirusInfecte()
                            player.health = Player.calculate_health(player)
                            wow_map.grid[player.location[0]][player.location[1]] = 2
                        break  #Exit the loop once a player is found.

                 # Check if a non player is present in the cell.
                for npc in npc_population:
                    if npc.location[0] == infect_player.location[0] + i and \
                            npc.location[1] == infect_player.location[1] + j:
                        #Infect the non-player if the square is occupied by a healthy non-player
                        if not npc.infected:
                            npc.infected = True
                            wow_map.grid[npc.location[0]][npc.location[1]] = 4
                        break  # Exit the loop once a non-player is found

def move_away_from_infected_zone(player):
    # Retrieve the coordinates of the player's current location
    current_x, current_y = player.location

    # Calculate the number of squares the player can move based on their speed
    movement_range = calculate_movement_range(player.genotype[6])

    # Find the direction opposite to the infected area
    opposite_direction_x = -1 if current_x < (wow_map.num_zones * wow_map.zone_size) / 2 else 1
    opposite_direction_y = -1 if current_y < (wow_map.num_zones * wow_map.zone_size) / 2 else 1

    # Calculate the new position by moving in the opposite direction
    new_x = current_x + (opposite_direction_x * movement_range)
    new_y = current_y + (opposite_direction_y * movement_range)

    # Check if the new position is valid (within map limits and not occupied)
    if 0 <= new_x < wow_map.num_zones * wow_map.zone_size and 0 <= new_y < wow_map.num_zones * wow_map.zone_size \
            and wow_map.grid[new_x][new_y] != 2:  # Check that the new position is not already occupied by an infected player
        # Update player position on map
        wow_map.grid[current_x][current_y] = 1  # Reset old position
        wow_map.grid[new_x][new_y] = 2  # Update new position
        player.move((new_x, new_y))  # Update player position in Player object

def move_towards_infected_zone(player):
    # Retrieve the coordinates of the player's current location
    current_x, current_y = player.location

    # Calculate the number of squares the player can move based on their speed
    movement_range = calculate_movement_range(player.genotype[6])

    # Find the direction to the infected area
    direction_x = -1 if current_x >= (wow_map.num_zones * wow_map.zone_size) / 2 else 1
    direction_y = -1 if current_y >= (wow_map.num_zones * wow_map.zone_size) / 2 else 1

    # Calculate the new position by moving in the direction of the infected area
    new_x = current_x + (direction_x * movement_range)
    new_y = current_y + (direction_y * movement_range)

    # Check if the new position is valid (within map limits and not occupied)
    if 0 <= new_x < wow_map.num_zones * wow_map.zone_size and 0 <= new_y < wow_map.num_zones * wow_map.zone_size \
            and wow_map.grid[new_x][new_y] != 2:  # Check that the new position is not already occupied by an infected player
        # Update player position on map
        wow_map.grid[current_x][current_y] = 1
        wow_map.grid[new_x][new_y] = 2
        player.move((new_x, new_y))

def calculate_movement_range(speed_genotype):

    movement_range = int(speed_genotype * 10)  #10 is the maximum number of squares on the map
    return movement_range

def move_away_from_populated_aera(player):

    current_x, current_y = player.location


    opposite_direction_x = -1 if current_x < (wow_map.num_zones * wow_map.zone_size) / 2 else 1
    opposite_direction_y = -1 if current_y < (wow_map.num_zones * wow_map.zone_size) / 2 else 1


    new_x = current_x + opposite_direction_x
    new_y = current_y + opposite_direction_y


    if 0 <= new_x < wow_map.num_zones * wow_map.zone_size and 0 <= new_y < wow_map.num_zones * wow_map.zone_size \
            and wow_map.grid[new_x][new_y] != 2:

        wow_map.grid[current_x][current_y] = 1
        wow_map.grid[new_x][new_y] = 2
        player.move((new_x, new_y))

def move_towards_populated_aera(player):

    current_x, current_y = player.location

    direction_x = -1 if current_x >= (wow_map.num_zones * wow_map.zone_size) / 2 else 1
    direction_y = -1 if current_y >= (wow_map.num_zones * wow_map.zone_size) / 2 else 1


    new_x = current_x + direction_x
    new_y = current_y + direction_y


    if 0 <= new_x < wow_map.num_zones * wow_map.zone_size and 0 <= new_y < wow_map.num_zones * wow_map.zone_size \
            and wow_map.grid[new_x][new_y] != 2:

        wow_map.grid[current_x][current_y] = 1
        wow_map.grid[new_x][new_y] = 2
        player.move((new_x, new_y))

def heals_player_nearby(player):
    for i in range(-1, 2):
        for j in range(-1, 2):

            if 0 <= player.location[0] + i < wow_map.num_zones * wow_map.zone_size and \
                    0 <= player.location[1] + j < wow_map.num_zones * wow_map.zone_size:

                for other_player in population:
                    if other_player.location[0] == player.location[0] + i and \
                            other_player.location[1] == player.location[1] + j:

                        if other_player.infected:
                            other_player.health = Player.calculate_health(other_player)
                            other_player.infected = False
                            wow_map.grid[other_player.location[0]][other_player.location[1]] = 3
                        break




def fitness(player):

    return sum(player.genotype)
def sort_population_by_fitness(population):
    return sorted(population, key=lambda x: fitness(x), reverse=True)
selection =population
def selection_3_groupe(population,groupe1_size):

    population = sort_population_by_fitness(population)
    groupe1 = population[:int(len(population)*groupe1_size)]
    groupe2 = population[int(len(population)*(groupe1_size)):int(len(population)*(1-groupe1_size))]
    groupe3 = population[int(len(population)*(1-groupe1_size)):]
    return groupe1, groupe2, groupe3
def croisement (groupe2):

    enfants = []
    for i in range(0, len(groupe2)-2, 2):
        parent1 = groupe2[i]
        parent2 = groupe2[i+1]

        enfant1 = Player(genotype_length)
        enfant2 = Player(genotype_length)

        enfant1.genotype = parent1.genotype[:4] + parent2.genotype[4:]
        enfant2.genotype = parent2.genotype[:4] + parent1.genotype[4:]

        enfant1.location = parent1.location
        enfant1.infected = parent1.infected
        enfant1.health = parent2.health
        enfant1.classplayer=parent2.classplayer

        enfant2.location = parent2.location
        enfant2.infected = parent2.infected
        enfant2.health = parent1.health
        enfant2.classplayer=parent1.classplayer




        enfants.append(enfant1)
        enfants.append(enfant2)
        remove(parent1)
        remove(parent2)

    return enfants

def mutation(groupe3):

    for player in groupe3:
        for i in range(len(player.genotype)):
            if random.random() < 0.1:

                if i == 5:
                    player.genotype[i] = round(random.random() * 0.6, 2)

                    player.health = Player.calculate_health(player)
                else:
                    player.genotype[i] = round(random.random(), 2)
    return groupe3

def remove(player):
    population.remove(player)
    wow_map.grid[player.location[0]][player.location[1]] = 1
    valid_positions.append(player.location)

    return population


def evolution(population, groupe1_size):
    groupe1, groupe2, groupe3 = selection_3_groupe(population, groupe1_size)
    enfants = croisement(groupe2)
    mutation(groupe3)
    population = groupe1 + groupe3 + enfants
    return population


def fill_population(population):
    if len(population) == length_population:
        return population

    for i in range(wow_map.num_zones * wow_map.zone_size):
        for j in range(wow_map.num_zones * wow_map.zone_size):
            if not wow_map.is_in_zone(i, j):
                wow_map.grid[i][j] = 0
            if wow_map.grid[i][j] == 1 and wow_map.is_in_zone(i, j):
                valid_positions.append((i, j))

    while len(population) < length_population:

        player = Player(genotype_length)

        player_position = get_valid_position()
        player.location = player_position
        wow_map.grid[player_position[0]][player_position[1]] = 2 if player.infected else 3
        population.append(player)

    return population



def reinitialise_map():

    for i in range(wow_map.num_zones * wow_map.zone_size):
        for j in range(wow_map.num_zones * wow_map.zone_size):
            if not wow_map.is_in_zone(i, j):
                wow_map.grid[i][j] = 0
            if wow_map.grid[i][j] == 1 and wow_map.is_in_zone(i, j):
                valid_positions.append((i, j))

    for player in population:
        wow_map.grid[player.location[0]][player.location[1]] = 2 if player.infected else 3
    for npc in npc_population:
        wow_map.grid[npc.location[0]][npc.location[1]] = 4 if npc.infected else 5

dict_lvl_die={'warrior':19,'paladin':20,'hunter':28,'rogue':34,'priest':38,'warlock':40,'mage':40,'shaman':21,'druid':23}


def step():

    random.shuffle(population)
    for player in population:
        if wow_map.grid[player.location[0]][player.location[1]] == 2:
            player.health -= 125
            #if the player level genotype[5]*100 is lower than dict_lvl_die[player.classplayer] he dies
            if player.genotype[5]*100 < dict_lvl_die[player.classplayer]:
                wow_map.grid[player.location[0]][player.location[1]] = 1
                valid_positions.append(player.location)
                remove(player)
                continue

           # if it drops below 0, it dies, we remove the population
            elif player.health <= 0:
                #we remove the player from the map
                wow_map.grid[player.location[0]][player.location[1]] = 1
                remove(player)
                continue
            infect_nearby_players_npc(player)
    for npc in npc_population:
        if wow_map.grid[npc.location[0]][npc.location[1]] == 4:
            infect_nearby_players_npc(npc)
    # players heal themselves or not thanks to the probability of genotype[4] (Treatment preference on self) they have genotype[4] chance of healing themselves
    for player in population:
        #if it's a paladin, druid, shaman or priest
        if player.classplayer in ['paladin','druid','shaman','priest']:
            if player.infected :
                if random.random() < player.genotype[4]:

                    player.health = Player.calculate_health(player)
                    player.infected = False
                    wow_map.grid[player.location[0]][player.location[1]] = 3
            #genotype[1]: Tendency to help other players (0 = low to 1 = high)
            if random.random() < player.genotype[1] :

                heals_player_nearby(player)

        #moving players based on their genotype
        #Creating a cumulative list of genotype values
        cumulative_genotype_values = [player.genotype[0], player.genotype[0] + player.genotype[3],
                                      player.genotype[0] + player.genotype[3] + player.genotype[7]]

        # Generation of a random number between 0 and the sum of genotype values
        random_value = random.random() * cumulative_genotype_values[-1]
        if random_value < cumulative_genotype_values[0]:
            #genome[0]: Tendency to escape from an infected area
            if random.random() < player.genotype[0]:  # Tendency to escape from an infected area
                move_away_from_infected_zone(player)
            else:
                move_towards_infected_zone(player)
        elif random_value < cumulative_genotype_values[1]:
            #genotype[3]: Tendency to explore infected areas (0 = low, 1 = high)
            if random.random() < player.genotype[3]:
                move_towards_infected_zone(player)
            else:
                move_away_from_infected_zone(player)
        else:
            #genotype[7]: Location (0 = go to populated places to 1 = go to places where there are no people)
            if random.random() < player.genotype[7]:
                move_away_from_populated_aera(player)
            else:
                move_towards_populated_aera(player)

wow_map.print_map()


. . . . . O . . . . . . . . . O . . . . . . . . . O . . . . 
. . O O O P O O P . . . O P O O O O O . . . O O O O O P O . 
. N O O O O O P O O . P O O O P O O P O . O O O O O O O O O 
. P O O O O O O O O . O O O O O O P O O . O O O O O O O O O 
. P P O O O P O O O . O O O O O O O P O . O O O O O O O O P 
P O O O P O O O O O O O O P O O O P O O O P O P O O O O O O 
. O O O O O O O O O . P O P P O O O O O . O O O O O O O O O 
. O O O O O O O O O . O P O O O P O O O . O O O O O O O P P 
. O P O O O O O P O . O O O O O O O P O . O O P P O O O O O 
. . P P O O O O O . . . O O O O O O O . . . O P P O O O O . 
. . . . . O . . . . . . . . . O . . . . . . . . . O . . . . 
. . O O O O O O O . . . P O O O O O P . . . O O O O O O O . 
. O O O O O O O O N . O O P O O O O O P . O O O O O O O O O 
. O O O O O O O P O . O P O O O O O O O . O O O O P P O O O 
. O O O P O O O O O . O O O O O O N O O . O O O O O P O O P 
O O O P O O O O O O O O O O O O O O O O O O O P P O P O O O 
. O O O O O O O O O . O 

Here is the map with the different players and the NPC.
"P" in red means it is an infected player, "P" in white means it is a non-infected player.
"N" in green means it is a non-infected NPC, and "N" in yellow means it is an infected NPC.

In [12]:
step()
wow_map.print_map()

. . . . . O . . . . . . . . . O . . . . . . . . . O . . . . 
. . O O O P O O P . P . O P O O O O O . . . O O O O O O O . 
P N O O O O O O O O . O O O O O O O O O . O O O O O O O O O 
. O O O O O O O O O . O O O P O O P O O . O O O O P O O O O 
. O O O O P O O O P . O O O O O O O P O . O P O O O O P O P 
O O P O O O O O O O P O O O O O O P O O O O O O O O O O O O 
. P O O O O O O P O . P O P O O O O O O . O O P O O O O O O 
. O O O O O O P O O . O O O O P O O O P . O O O O O O O O P 
. O O O O O O P O O . O O O O O O O O O . O O P P P O P O O 
. . P P O O P O O . . P O O P O O O O P P . O O O O O O O . 
. . . . . O . . P . . P . . . O . . . . . . . . . O . . . . 
. . O O O O O O O . . . O O P O O O O . . . O O O O O O O . 
. O O O O O O P O N . O O O O O O O O O . O O O O O P O O O 
. O O P O O O O O O . P O O O O O O O O P O O O O O O O O O 
. O O O O O O O O O . O P O O O O N O O . O O O O O O O O P 
O O O O O O O O O O O O O O O O O O O O O O O O O P P O O O 
. O O O O O O O O O P O 

Here is the map after one step. We can see that the majority of players are infected.

In [13]:
for _ in range(20):
  step()
wow_map.print_map()

P . . . O O . . . . . . . . . O . . . . P . . . . O P O O O 
. O O O O O P O O . O . O O O O O O O O . . O O P O O O O P 
O O O O P O O O O O . O O O O O O O O O . O O O O O O O O O 
. O O O O O O O O O O O O P O O O O O O . O O O O P O O O O 
. O O O O P O O O O . O O O O O O O O O . O O O O O O O O P 
O O O O O P O O O O O O O O O O O O O O O O O O O O O O O O 
. O O O O O O O O O P O O O O O P O O O . P O O O O O O O O 
. O P O O O P O O O . O O O O O O O O O O P P O O O O O O O 
. O O O O O O O O O O O O O P O O O O O O P O O O O O O O O 
. . O O O O O O O O . O O O O O O O O O O O O O O O O O O O 
. . . . . O . O O O O O O O O O . O O P . . O . . O . . O . 
. . O O O O O O O . O O O O O O O P P . . P O O O O O O P . 
. O O O O O O O O O . O O O O O O P O P O O O O O O O O O O 
. O O O O O O O O O O P O P O O O O P O O O O O O O O O O O 
. O O O O O O O O O . O O O O O O O P O O P O O O O O O O O 
O O O O O O O O O O O O P O O O O O O O O O O O O O O O O O 
. O O O O O O O O P O P 

After 20 steps, almost everyone is infected.

In [14]:
for i in range(100):
            for j in range(20):
                step()
            population = evolution(population, 0.5)
            reinitialise_map()
            population = fill_population(population)

In [15]:
wow_map.print_map()

. . . . . O . . . . . . . . . O . . . . . . . . . O . . . . 
. . O O O O O O O . . . O O O O O O O . . . O O O O O O P . 
. N P P O O O O O O . O O O O O O O O O . O O O O O O O O O 
. O O P O O O O O O . O O O O O O O O O . O O O O P P O O O 
. O O O P O O O O O . O O O O O O O O O . O O O O P P O O O 
O O O O O P O O O O O O O O O O O O O O O O O O P O O O O O 
. O O O O O P O O O . O O O O O O O O O . O O P O O O O O O 
. O O O O O O P O O . O O O O O O O O O . O O O O O P O O O 
. O O O O O O O P O . O O O O O O O O P . P O O O O O O O O 
. . O O O O O O O . . . O O O O O O O . . . O O O O O O O . 
. . . . . O . . . . . . . . . O . . . . . . . . . O . . . . 
. . O O O O O O O . . . O O O P O O P . . . O O O O O O O . 
. O O O O O O O O N . O P O O O O O O O . O O O O O O O O O 
. O O O O O O O O O . O O P O O P O O O . O O O O O O O O O 
. O O O O O O O O O . O O O P P O N O O . O O O O O O O O O 
O O O O O O O O P O O O O O P P O O O O O O O O O O O O O O 
. O O O O O O O O O . O 

After 100 sets of 20 steps and evolutions, we end up with a population where there are not too many infected individuals: the infection has been slowed down.

After a certain number of steps, we will leverage genetic algorithm methods.
Firstly, we will sort the population according to the fitness function, which will sum the elements of a genotype.
Then, we will create three groups within this sorting. The first group will be the top of the sorted list, which we will keep as it is. The second group will undergo a two-point crossover. Finally, the third group will undergo a crossover where each gene has a probability of 0.2 of being modified.
We fill the population with random individuals, and then we repeat the cycle to obtain a population where there are few or no infected individuals, thus reducing the spread of the virus

In [16]:
afficher_player()

[0.2, 0.98, 0.97, 0.88, 0.97, 0.95, 0.49, 0.99]
Santé:  14168.449999999999
role:  warrior
-------------------
[0.72, 0.8, 0.82, 0.63, 0.59, 0.96, 0.82, 0.82]
Santé:  13754.879999999997
role:  paladin
-------------------
[0.87, 0.99, 0.35, 0.85, 0.98, 0.92, 0.32, 0.81]
Santé:  11627.36
role:  druid
-------------------
[0.61, 0.79, 0.57, 0.9, 0.44, 0.99, 0.8, 0.9]
Santé:  14180.970000000001
role:  paladin
-------------------
[0.94, 0.32, 0.94, 0.66, 0.89, 0.96, 0.8, 0.4]
Santé:  9575.0
role:  hunter
-------------------
[0.73, 0.88, 0.88, 0.6, 0.47, 0.67, 0.82, 0.83]
Santé:  10204.17
role:  warrior
-------------------
[0.81, 0.81, 0.56, 0.55, 0.58, 0.71, 0.9, 0.94]
Santé:  6825.0
role:  hunter
-------------------
[0.89, 0.25, 0.71, 0.81, 0.94, 0.98, 0.74, 0.53]
Santé:  14869.98
role:  warrior
-------------------
[0.91, 1.0, 0.98, 0.76, 0.28, 0.46, 0.55, 0.91]
Santé:  6528.379999999999
role:  paladin
-------------------
[0.59, 0.5, 0.97, 0.8, 0.62, 0.77, 0.72, 0.88]
Santé:  11056.31
role: 

We can see that there are practically no more infected individuals.

In [17]:
for i in range(100):
            for j in range(20):
                step()
            population = evolution(population, 0.2)
            reinitialise_map()
            population = fill_population(population)

In [18]:
afficher_player()

[0.85, 0.91, 0.69, 0.91, 0.74, 0.85, 0.94, 1.0]
Santé:  12192.55
role:  paladin
-------------------
[0.95, 0.8, 0.87, 0.99, 0.99, 0.75, 0.48, 0.91]
Santé:  9251.0
role:  druid
-------------------
[0.91, 1.0, 0.98, 0.76, 0.72, 0.75, 0.65, 0.93]
Santé:  9501.0
role:  druid
-------------------
[0.95, 0.77, 0.62, 0.76, 0.95, 0.68, 0.91, 0.88]
Santé:  9202.08
role:  shaman
-------------------
[0.93, 0.46, 0.95, 0.79, 0.86, 0.96, 0.79, 0.74]
Santé:  12816.76
role:  shaman
-------------------
[1.0, 0.87, 0.64, 0.86, 0.57, 0.67, 0.99, 0.87]
Santé:  10204.17
role:  warrior
-------------------
[0.84, 0.9, 0.96, 0.68, 0.9, 0.43, 0.97, 0.78]
Santé:  5373.44
role:  druid
-------------------
[0.72, 0.97, 0.76, 0.93, 0.96, 0.7, 0.97, 0.43]
Santé:  8750.6
role:  druid
-------------------
[0.94, 0.93, 0.5, 0.88, 0.88, 0.53, 0.89, 0.82]
Santé:  5150.0
role:  hunter
-------------------
[0.95, 0.8, 0.68, 0.96, 0.83, 0.68, 0.44, 0.99]
Santé:  9202.08
role:  shaman
-------------------
[0.98, 0.62, 0.83, 0.5